<a href="https://colab.research.google.com/github/SvA1/Compling/blob/master/HW5_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pymorphy2

     |████████████████████████████████| 51kB 4.0MB/s 
     |████████████████████████████████| 7.1MB 14.2MB/s 


In [3]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 9.6MB/s 
     |████████████████████████████████| 552kB 45.6MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=88935fd6a7ec365e78ab210d215a4bd15f47f113544353cf1e09cbc50a17dd06
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=4c0c972212b4e19228fe4f0f869a0b0bb5967f0b2ea9b425485aef2ef90157ff
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [0]:
import re
import json
import gensim
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim
import string
from collections import Counter

morph = MorphAnalyzer()

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
!wget https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/habr_texts.txt.zip
!unzip habr_texts.txt.zip

In [0]:
with open('habr_texts.txt', encoding='utf-8') as file:
    source = file.read().splitlines()

**1. Нормализация**

In [0]:
stop_words = set(stopwords.words('russian')) | {'gt',}

def clean_text(text):
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'{[^}]+}', ' ', text)
    text = re.sub(r'\&[a-z]+\;', ' ', text)
    text = text.replace('\t', ' ')
    return text

def tokenize(text):
    return re.findall(r'[А-ЯЁа-яё]+', text)

In [0]:
def opt_normalize(texts, top=None):
    uniq = Counter()
    for text in texts:
        uniq.update(text)
    
    norm_uniq = {word:morph.parse(word)[0].normal_form for word, _ in uniq.most_common(top)}
    
    norm_texts = []
    for text in texts:
        
        norm_words = [norm_uniq.get(word) for word in text]
        norm_words = [word for word in norm_words if word and word not in stop_words]
        norm_texts.append(norm_words)
        
    return norm_texts

**2. Добавление нграммов**

In [0]:
texts = opt_normalize([tokenize(clean_text(text)) for text in source])

In [0]:
ph = gensim.models.Phrases(texts, scoring='npmi', min_count=8, threshold=0.2)
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

**3. Словарь**

In [0]:
dictionary = gensim.corpora.Dictionary(ngrammed_texts)

In [0]:
dictionary.filter_extremes(no_above=0.1, no_below=15)
dictionary.compactify()

In [0]:
corpus = [dictionary.doc2bow(text) for text in ngrammed_texts]

**4. LDA модели**

In [0]:
lda1 = gensim.models.LdaMulticore(corpus, id2word=dictionary, eval_every=1, num_topics=10, passes=6, alpha='asymmetric')
lda2 = gensim.models.LdaModel(corpus, id2word=dictionary, eval_every=1, num_topics=10, passes=3, alpha='auto')
lda3 = gensim.models.LdaMulticore(corpus, id2word=dictionary, eval_every=2, num_topics=20, passes=5)

In [38]:
lda1.print_topics(10)

[(0,
  '0.007*"заказчик" + 0.006*"мозг" + 0.005*"выражение" + 0.005*"дефект" + 0.004*"стикер" + 0.004*"доклад" + 0.004*"робот" + 0.004*"ребёнок" + 0.004*"лицо" + 0.003*"усилитель"'),
 (1,
  '0.004*"производство" + 0.004*"звук" + 0.004*"двигатель" + 0.003*"синтезатор" + 0.002*"изделие" + 0.002*"поезд" + 0.002*"фотография" + 0.002*"представитель" + 0.002*"советский" + 0.002*"ядро"'),
 (2,
  '0.006*"цикл" + 0.005*"узел" + 0.004*"процессор" + 0.004*"браузер" + 0.003*"инфраструктура" + 0.003*"переменный" + 0.003*"кластер" + 0.003*"контент" + 0.003*"виртуальный" + 0.003*"задержка"'),
 (3,
  '0.009*"массив" + 0.006*"кампания" + 0.005*"ключ" + 0.004*"репликация" + 0.004*"шаблон" + 0.003*"электроника" + 0.003*"бизнес_логика" + 0.003*"диск" + 0.003*"олимпиада" + 0.003*"загрузка"'),
 (4,
  '0.007*"вакансия" + 0.005*"камера" + 0.004*"яркость" + 0.004*"датчик" + 0.004*"дом" + 0.003*"смартфон" + 0.003*"стандарт" + 0.003*"инвестиция" + 0.003*"офис" + 0.003*"свет"'),
 (5,
  '0.015*"игрок" + 0.007*"уда

In [39]:
lda2.print_topics(10)

[(0,
  '0.008*"массив" + 0.007*"узел" + 0.006*"ядро" + 0.005*"репликация" + 0.005*"раздел" + 0.005*"контейнер" + 0.005*"ос" + 0.005*"порт" + 0.004*"загрузка" + 0.004*"прибор"'),
 (1,
  '0.007*"браузер" + 0.004*"студент" + 0.004*"контент" + 0.004*"курс" + 0.004*"релиз" + 0.004*"бот" + 0.004*"программирование" + 0.003*"аудитория" + 0.003*"перевод" + 0.003*"чек_лист"'),
 (2,
  '0.017*"услуга" + 0.014*"вакансия" + 0.011*"домен" + 0.008*"дефект" + 0.007*"трафик" + 0.007*"партнёр" + 0.006*"инцидент" + 0.004*"мониторинг" + 0.004*"профиль" + 0.004*"хостинг"'),
 (3,
  '0.007*"робот" + 0.006*"электроника" + 0.006*"китай" + 0.006*"олимпиада" + 0.005*"двигатель" + 0.004*"южный_корея" + 0.004*"закон" + 0.003*"российский" + 0.003*"заявить" + 0.003*"китайский"'),
 (4,
  '0.005*"цикл" + 0.005*"атрибут" + 0.005*"контекст" + 0.005*"переменный" + 0.004*"шаблон" + 0.004*"вычисление" + 0.004*"логика" + 0.004*"поль" + 0.003*"дерево" + 0.003*"возвращать"'),
 (5,
  '0.010*"боль" + 0.008*"мозг" + 0.005*"клетка

In [40]:
lda3.print_topics(20)

[(0,
  '0.008*"цифровой" + 0.007*"бот" + 0.006*"публикация" + 0.006*"шаблон" + 0.005*"распространение" + 0.005*"твит" + 0.005*"набор_дать" + 0.005*"лицо" + 0.004*"контакт" + 0.004*"директива"'),
 (1,
  '0.034*"кампания" + 0.015*"объявление" + 0.014*"конвертер" + 0.012*"таймер" + 0.012*"заголовок" + 0.010*"доклад" + 0.009*"перенос" + 0.008*"ключ" + 0.008*"фоновый" + 0.008*"бюджет"'),
 (2,
  '0.017*"ядро" + 0.016*"процессор" + 0.014*"студент" + 0.013*"батарея" + 0.013*"аккумулятор" + 0.010*"байт" + 0.008*"отметка" + 0.008*"линия" + 0.007*"контрольный" + 0.007*"ёмкость"'),
 (3,
  '0.015*"заказчик" + 0.014*"игрок" + 0.008*"услуга" + 0.007*"партнёр" + 0.006*"ожидание" + 0.005*"играть" + 0.005*"респондент" + 0.005*"трафик" + 0.005*"инцидент" + 0.004*"аудитория"'),
 (4,
  '0.014*"стикер" + 0.010*"синтезатор" + 0.009*"спецификация" + 0.008*"китай" + 0.007*"логин" + 0.007*"загрузчик" + 0.007*"доска" + 0.006*"браузер" + 0.006*"шаблон" + 0.006*"фильтр"'),
 (5,
  '0.021*"массив" + 0.010*"доклад" +

**5. 3 темы для модели**

На мой взгляд все модели справились одинаково плохо. Тем не менее, наиболее приемлимый результат у второй модели.

Тема, связанная с МО

>   '0.008*"массив" + 0.007*"узел" + 0.006*"ядро" + 0.005*"репликация" + 0.005*"раздел" + 0.005*"контейнер" + 0.005*"ос" + 0.005*"порт" + 0.004*"загрузка" + 0.004*"прибор"'

Тема, связанная с роботами

>   '0.007*"робот" + 0.006*"электроника" + 0.006*"китай" + 0.006*"олимпиада" + 0.005*"двигатель" + 0.004*"южный_корея" + 0.004*"закон" + 0.003*"российский" + 0.003*"заявить" + 0.003*"китайский"'

Что-то про игры
>  '0.013*"игрок" + 0.004*"играть" + 0.003*"продажа" + 0.003*"инвестиция" + 0.002*"игровой" + 0.002*"цифровой" + 0.002*"дизайнер" + 0.002*"публикация" + 0.002*"интерес" + 0.002*"ия"'






**6. TfIdf**

In [0]:
tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary)
corpus_tfidf = tfidf[corpus]

In [0]:
lda1_tfidf = gensim.models.LdaMulticore(corpus_tfidf, id2word=dictionary, eval_every=1, num_topics=10, passes=6, alpha='asymmetric')
lda2_tfidf = gensim.models.LdaModel(corpus_tfidf, id2word=dictionary, eval_every=1, num_topics=10, passes=3, alpha='auto')
lda3_tfidf = gensim.models.LdaMulticore(corpus_tfidf, id2word=dictionary, eval_every=2, num_topics=20, passes=5)

In [45]:
lda1_tfidf.print_topics(10)

[(0,
  '0.001*"браузер" + 0.001*"услуга" + 0.001*"игрок" + 0.001*"доклад" + 0.001*"контейнер" + 0.001*"заказчик" + 0.001*"контекст" + 0.001*"бот" + 0.001*"программирование" + 0.001*"контент"'),
 (1,
  '0.000*"подложка" + 0.000*"паяльник" + 0.000*"регистр" + 0.000*"шт" + 0.000*"астероид" + 0.000*"плис" + 0.000*"закрытый_ключ" + 0.000*"объектив" + 0.000*"узор" + 0.000*"битый"'),
 (2,
  '0.000*"ибп" + 0.000*"бот" + 0.000*"игрок" + 0.000*"пароль" + 0.000*"товар" + 0.000*"книга" + 0.000*"сигнал" + 0.000*"доклад" + 0.000*"шаблон" + 0.000*"запоминание"'),
 (3,
  '0.000*"сборка" + 0.000*"виртуальный_машина" + 0.000*"агент" + 0.000*"секция" + 0.000*"ядро" + 0.000*"ключ" + 0.000*"указатель" + 0.000*"анимация" + 0.000*"опция" + 0.000*"облако"'),
 (4,
  '0.000*"змейка" + 0.000*"звук" + 0.000*"игрок" + 0.000*"абонент" + 0.000*"ботнет" + 0.000*"зуб" + 0.000*"контекст" + 0.000*"вагон" + 0.000*"письмо" + 0.000*"оператор"'),
 (5,
  '0.000*"кэш" + 0.000*"диск" + 0.000*"массив" + 0.000*"охрана" + 0.000*"

In [46]:
lda2_tfidf.print_topics(10)

[(0,
  '0.001*"система_видеонаблюдение" + 0.000*"частное_облако" + 0.000*"снаряд" + 0.000*"шейдёр" + 0.000*"протез" + 0.000*"место_проведение" + 0.000*"веб_камера" + 0.000*"эксплойт" + 0.000*"датчик" + 0.000*"нота"'),
 (1,
  '0.009*"усилитель" + 0.002*"бактерия" + 0.001*"паяльник" + 0.001*"полупроводниковый" + 0.001*"геном" + 0.001*"составлять_миллиард" + 0.001*"антибиотик" + 0.001*"препарат" + 0.001*"вич" + 0.000*"стабилизатор"'),
 (2,
  '0.007*"клетка" + 0.006*"ген" + 0.002*"генетический" + 0.002*"белка" + 0.002*"мутация" + 0.002*"клеточный" + 0.002*"днк" + 0.001*"молекула" + 0.001*"биолог" + 0.001*"болезнь"'),
 (3,
  '0.010*"стикер" + 0.006*"двигатель" + 0.006*"километр_число" + 0.006*"протон" + 0.004*"тоннель" + 0.004*"роскосмос" + 0.004*"пуск" + 0.003*"половый" + 0.003*"орбита" + 0.003*"ракета_носитель"'),
 (4,
  '0.001*"браузер" + 0.001*"услуга" + 0.001*"доклад" + 0.001*"контейнер" + 0.001*"контекст" + 0.001*"программирование" + 0.001*"шаблон" + 0.001*"контент" + 0.001*"бот" + 0.

In [47]:
lda3_tfidf.print_topics(20)

[(0,
  '0.009*"контекст" + 0.008*"логин" + 0.007*"шаблон" + 0.007*"бизнес_логика" + 0.006*"перевод" + 0.006*"тег" + 0.004*"блокировать" + 0.004*"туториал" + 0.004*"виджет" + 0.004*"граф"'),
 (1,
  '0.016*"законопроект" + 0.014*"реле" + 0.007*"регулирование" + 0.006*"качать" + 0.005*"щит" + 0.004*"микросервис" + 0.004*"кэширование" + 0.004*"закон" + 0.004*"стабилизатор" + 0.004*"тасс"'),
 (2,
  '0.018*"синтезатор" + 0.012*"загрузчик" + 0.011*"хук" + 0.006*"линия" + 0.005*"советский" + 0.005*"связующий" + 0.005*"спецификация" + 0.004*"неисправный" + 0.003*"значительный_степень" + 0.002*"сподвигнуть"'),
 (3,
  '0.006*"исполнительный_директор" + 0.006*"текстура" + 0.006*"переадресация" + 0.006*"сделка" + 0.004*"республика" + 0.004*"губа" + 0.004*"холдинг" + 0.003*"отклонить" + 0.003*"софтверный" + 0.003*"туннель"'),
 (4,
  '0.006*"студент" + 0.006*"вакансия" + 0.004*"олимпиада" + 0.004*"электроника" + 0.004*"чек_лист" + 0.004*"лекция" + 0.004*"курс" + 0.003*"привязка" + 0.003*"фсб" + 0.002

**7. Анализ**

Заметила, что в tfidf моделях извлеклось большое количество словосочетаний, и из-за этого легче понять тему, например:

> '0.001*"система_видеонаблюдение" + 0.000*"частное_облако" + 0.000*"снаряд" + 0.000*"шейдёр" + 0.000*"протез" + 0.000*"место_проведение" + 0.000*"веб_камера" + 0.000*"эксплойт" + 0.000*"датчик" + 0.000*"нота"'

Понятно, что тема скорее всего про вебкамеры\видеонаблюдение

Особенно выделяются словосочетания в третьей модели: некоторые словосочетания практически готовые темы
> "загрузить_файл" + 0.006*"паттерн" + 0.005*"система_безопасность" + 0.004*"скриптовый_язык


Вторая модель

Законопроект,затрагивающий беспилотные автомобили (непонятно, причем тут роскосмос и небоскрёб)
> '0.007*"законопроект" + 0.006*"фсб" + 0.002*"беспилотный_автомобиль" + 0.001*"плазма" + 0.001*"заместитель" + 0.001*"роскосмос" + 0.001*"небоскрёб" + 0.001*"слово_представитель" + 0.001*"протон" + 0.001*"яркость"'

Космос - это тема не была выделена в модели LDA
> '0.002*"дефект" + 0.000*"телеметрия" + 0.000*"меркурий" + 0.000*"единственный_вариант" + 0.000*"планета" + 0.000*"сатурн" + 0.000*"орбита" + 0.000*"космонавт" + 0.000*"симметрия" + 0.000*"небесный_тело"'

Генетика\биология - тоже была не совсем явно выделена в предыдущей модели
> '0.007*"клетка" + 0.006*"ген" + 0.002*"генетический" + 0.002*"белка" + 0.002*"мутация" + 0.002*"клеточный" + 0.002*"днк" + 0.001*"молекула" + 0.001*"биолог" + 0.001*"болезнь"'

Выделилась тема МО и нейросети, в LDA такого не было 
>'0.000*"законопроект" + 0.000*"алиас" + 0.000*"нейронный_сеть" + 0.000*"датасет" + 0.000*"нейрон" + 0.000*"автопилот" + 0.000*"унылый" + 0.000*"глубокий_обучение" + 0.000*"весы" + 0.000*"перечисление"')
В третьей модели еще выделилась тема майнинга.

В целом, кажется, что с использованием tfidf темы выделять проще






**8. NMF + Tfidf Vectorizer**

In [0]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

In [0]:
str_texts = [' '.join(text) for text in texts]

In [0]:
vectorizer = TfidfVectorizer(max_features=800, min_df=12, max_df=0.4, ngram_range=(1,2))
X = vectorizer.fit_transform(str_texts)

In [0]:
model = NMF(n_components=30)

In [28]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=30, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [0]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [0]:
feat_names = vectorizer.get_feature_names()

In [31]:
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 ваш  знать  сказать  говорить
1 файл  папка  скрипт  команда
2 игра  игрок  игровой  играть
3 функция  значение  переменный  массив
4 устройство  смартфон  телефон  производитель
5 приложение  мобильный  разработка  платформа
6 дата центр  центр  дата  оборудование
7 рубль  товар  цена  рынок
8 учёный  мозг  исследование  научный
9 уязвимость  атака  безопасность  защита
10 сайт  страница  браузер  реклама
11 доклад  конференция  разработчик  видео
12 сеть  интернет  трафик  связь
13 код  библиотека  ошибка  исходный
14 книга  часы  читать  электронный
15 запрос  база  таблица  база дать
16 язык  программирование  программист  программа
17 объект  класс  метод  тип
18 элемент  компонент  блок  свойство
19 память  процессор  диск  ядро
20 тест  тестирование  тестовый  производительность
21 сигнал  звук  частота  диапазон
22 камера  видео  изображение  смартфон
23 робот  автомобиль  машина  ребёнок
24 сервер  виртуальный  машина  настройка
25 бот  сообщение  канал  команда
26 ключ  сер

In [32]:
vectorizer = TfidfVectorizer(max_features=500, min_df=7, max_df=0.3, ngram_range=(1,3))
X = vectorizer.fit_transform(str_texts)
model = NMF(n_components=20)
model.fit(X)
feat_names = vectorizer.get_feature_names()
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 жизнь  друг  думать  вообще
1 игра  игрок  игровой  играть
2 файл  строка  пароль  формат
3 сервер  домен  настройка  адрес
4 устройство  мобильный  производитель  управление
5 значение  переменный  массив  строка
6 браузер  веб  страница  библиотека
7 программа  компьютер  процессор  программный
8 продукт  мобильный  рынок  дизайн
9 сеть  интернет  трафик  связь
10 сайт  страница  домен  контент
11 тест  тестирование  ошибка  библиотека
12 объект  свойство  свет  пространство
13 запрос  таблица  ответ  страница
14 модель  изображение  обучение  алгоритм
15 сервис  облачный  сообщение  платформа
16 язык  программирование  программист  библиотека
17 бот  сообщение  канал  игрок
18 рубль  российский  цена  россия
19 элемент  компонент  свойство  состояние
20 книга  читать  часы  электронный
21 сотрудник  бизнес  рабочий  документ
22 класс  метод  библиотека  реализация
23 виртуальный  машина  диск  память
24 конференция  участник  видео  тема
25 центр  дата  оборудование  инфраструктур

In [33]:
vectorizer = TfidfVectorizer(max_features=800, min_df=17, max_df=0.3, ngram_range=(1,3))
X = vectorizer.fit_transform(str_texts)
model = NMF(n_components=20)
model.fit(X)
feat_names = vectorizer.get_feature_names()
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 вообще  думать  жизнь  ничто
1 игра  игрок  игровой  играть
2 файл  папка  скрипт  строка
3 элемент  значение  компонент  блок
4 устройство  смартфон  мобильный  телефон
5 продукт  мобильный  дизайн  инструмент
6 космический  аппарат  земля  станция
7 язык  программирование  программист  программа
8 сеть  интернет  трафик  связь
9 сервер  домен  настройка  адрес
10 доклад  конференция  тема  участник
11 сайт  страница  браузер  веб
12 тест  тестирование  тестовый  ошибка
13 запрос  таблица  строка  ответ
14 сигнал  частота  диапазон  канал
15 бот  скрипт  канал  друг
16 изображение  обучение  алгоритм  модель
17 класс  метод  библиотека  реализация
18 база  база дать  таблица  запись
19 печать  мм  материал  температура
20 товар  скидка  магазин  цена
21 дата центр  центр  дата  оборудование
22 сотрудник  бизнес  ит  рабочий
23 книга  часы  профессиональный  читать
24 робот  ребёнок  датчик  движение
25 учёный  мозг  исследование  научный
26 виртуальный  машина  реальность  резервный

In [34]:
vectorizer = TfidfVectorizer(max_features=100, min_df=5, max_df=0.4, ngram_range=(1,4))
X = vectorizer.fit_transform(str_texts)
model = NMF(n_components=20)
model.fit(X)
feat_names = vectorizer.get_feature_names()
top_words = model.components_.argsort()[:,:-5:-1]

for i in range(top_words.shape[0]):
    words = [feat_names[j] for j in top_words[i]]
    print(i, "  ".join(words))

0 код  ошибка  программа  тест
1 игра  разработчик  разработка  знать
2 тип  класс  метод  интерфейс
3 значение  состояние  строка  день
4 элемент  список  страница  запись
5 устройство  управление  уровень  настройка
6 файл  модуль  добавить  параметр
7 приложение  версия  разработчик  разработка
8 сайт  страница  строка  интернет
9 сервер  клиент  сервис  настройка
10 функция  параметр  уровень  строка
11 сеть  интерфейс  связь  интернет
12 модель  метод  уровень  модуль
13 точка  находиться  скорость  область
14 язык  программа  слово  технология
15 команда  разработка  разработчик  программа
16 запрос  база  запись  сервис
17 объект  метод  класс  инструмент
18 ваш  продукт  говорить  клиент
19 число  программа  строка  далее


**9. Темы лучшей NMF модели. Сравнение с LDA**

Бросаетсяв глаза, что темы теперь более четко разделены. 

Тем не менее, например, учитывая специфику текстлв, возможно стоило в стоп-слова включать "функция", "параметр", "инструмент" и т.п., потому что они встречаются часто, но редко несут ососбую смысловую нагрузку.
На мой взгляд, лучше всего справилась вторая модель.

Космос
> космический  аппарат  земля  станция

Настройка сервера
> сервер  домен  настройка  адрес

Базы данных
> запрос  таблица  строка  ответ

Информационная безопасность
>уязвимость  атака  безопасность  защита

